In [11]:
import simplekml
import teradata
import pandas as pd
from shapely.geometry import Polygon, Point

### Load from Teradata

In [3]:
#Teradata
password = 'um6lrq9ecek'

udaExec = teradata.UdaExec (appName="HelloWorld", version="1.0",
                            logConsole=False)

session = udaExec.connect(method="odbc", system='td2800.corp.tele2.ru', username ="ZMITROCHENKO_EA", password=password,
                          driver="Teradata", charset='utf8')

In [ ]:
%%time
df = pd.read_sql(
"""
SEL a.reg_name, a.key_name NAME_TAG, a.OSM_ID, a.WM_ID, gks_pop,
CASE WHEN c.WM_ID IS NOT NULL THEN c.WM_POLYGON.ST_AsText() ELSE b.MULTIPOLYGON.ST_AsText() END b
              FROM  UAT_DM.gi_geo_proto_3 a
              LEFT JOIN UAT_DM.OSM_RUSSIA_MULTIPOLYGONS b ON a.OSM_ID=b.OSM_ID
              LEFT JOIN UAT_DM.WM_PLACES c ON a.WM_ID=c.WM_ID
              WHERE 1=1
              and b.NAME_TAG IS NOT NULL
              ORDER BY gks_pop DESC
"""
, con=session)
#and (a.reg_name LIKE '%Дагестан%' or a.reg_name LIKE '%Орловская%')

In [287]:
test = df[df.OSM_ID == '7677449']

In [339]:
%%time
poly_list = []
for index, row in df.iterrows():
    #print('old', index)
    if 'MULTIPOLYGON' in row['b']:
        POLYGONS_OSM = row['b'].replace('MULTIPOLYGON (((', '').replace(')))', '').split(')),((')#.replace(',', '*')
        #print(POLYGONS_OSM[1])
        
#         POLYGONS_OSM = POLYGONS_OSM.split('-')
#         print('--',type(POLYGONS_OSM[0]))
        
        
        for j in range(len(POLYGONS_OSM)):
            POLYGON_OSM = POLYGONS_OSM[j].split(',')
            
            pointList = []
            for i in range(len(POLYGON_OSM)):
                x = float(POLYGON_OSM[i].split(' ')[0])
                y = float(POLYGON_OSM[i].split(' ')[1])
                pointList.append((x,y))
            poly_list.append(pointList)        
        

    else:
        POLYGON_OSM = row['b'].replace('POLYGON ((', '').replace('))', '')
        POLYGON_OSM = POLYGON_OSM.replace(')', '').replace('(', '').split(',')
        #print('new', POLYGON_OSM)
        pointList = []

        for i in range(len(POLYGON_OSM)):

            x = float(POLYGON_OSM[i].split(' ')[0])
            y = float(POLYGON_OSM[i].split(' ')[1])
            pointList.append((x,y))
        poly_list.append(pointList)

Wall time: 14 s


In [337]:
#poly_list

In [290]:
#df.head(455)

### Convert Polygon

In [342]:
# %%time
# poly_list = []
# for index, row in df.iterrows():
#     print('old', index)
#     POLYGON_OSM = row['b'].replace('POLYGON ((', '').replace('))', '')
#     POLYGON_OSM = POLYGON_OSM.replace(')', '').replace('(', '').split(',')
#     #print('new', POLYGON_OSM)
#     pointList = []
    
#     for i in range(len(POLYGON_OSM)):
        
#         x = float(POLYGON_OSM[i].split(' ')[0])
#         y = float(POLYGON_OSM[i].split(' ')[1])
#         pointList.append((x,y))
#     poly_list.append(pointList)
    
#df = df.drop('b', axis=1)
df['POLYGON'] = poly_list

ValueError: Length of values does not match length of index

In [343]:
for index, row in df.iterrows():
    if 'MULTIPOLYGON' in row['b']:
        print(row)

KeyError: 'b'

In [344]:
df.head(2)

,reg_name,NAME_TAG,OSM_ID,wm_id,gks_pop
0,Москва,москва,2555133,None,12330126.0
1,Москва,москва,2555133,None,12330126.0


### Create KML

In [269]:
#POLYGON

In [274]:
kml = simplekml.Kml()
for level in df.loc[:,u"reg_name"].unique():
    print(level)
    fol = kml.newfolder(name=level)
    df_s = df.loc[df[u"reg_name"]==level,:].iloc[:,0:len(df)]
    
    for index, row in df_s.iterrows():
        NAME_TAG = row['NAME_TAG']
        OSM_ID = row['OSM_ID']
        WM_ID = row['wm_id']
        gks_pop = row['gks_pop']
        POLYGON = row['POLYGON']
        if OSM_ID != None:
            decr = 'OSM_ID = '+str(OSM_ID)+', gks_pop = '+str(gks_pop)[:-2]
            #print(NAME_TAG)
            #print(decr)
        else:
            decr = 'WM_ID = '+str(WM_ID)+', gks_pop = '+str(gks_pop)[:-2]


        #print(type(POLYGON[0]))
        pol = fol.newpolygon(name=NAME_TAG, description=decr,
                         outerboundaryis=POLYGON)
        pol.style.linestyle.color = simplekml.Color.blue
        pol.style.linestyle.width = 1
        pol.style.polystyle.color = simplekml.Color.changealphaint(100, simplekml.Color.blue)

Республика Дагестан                     
Орловская область                       


In [275]:
kml.save("C:\\Users\\evgenia.zmitrochenko\\Documents\\Task\\All_region.kml")